<a href="https://colab.research.google.com/github/sameer0541/text_toxicity/blob/main/Text_toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
from tensorflow.keras.layers import TextVectorization

In [3]:
data = pd.read_csv('/content/train.csv')

In [4]:
data.head().columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [5]:
X = data['comment_text']
Y = data.iloc[:,2:]

In [6]:
vectorizer = TextVectorization(max_tokens=200000,output_sequence_length=1000)

In [7]:
vectorizer.adapt(X.values)

In [8]:
vectorized_text = vectorizer(X.values)

In [9]:
vectorized_text.shape

TensorShape([159571, 1000])

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [11]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

#model

In [12]:
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Embedding,Dropout
from tensorflow.keras.models import Sequential

In [13]:
model = Sequential()

In [14]:
model.add(Embedding(200001,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [16]:
model.compile(optimizer='Adam',loss=tf.keras.losses.BinaryCrossentropy)

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
hist = model.fit(train,epochs = 1,validation_data = val)

6981/6981 ━━━━━━━━━━━━━━━━━━━━ 416s 59ms/step - loss: 0.0843 - val_loss: 0.0461


In [24]:
input = vectorizer('You freaking suck! I am going to hit you.')

In [25]:
yhat = model.predict(np.expand_dims(input,0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [26]:
yhat

array([[0.9891676 , 0.32905453, 0.9383692 , 0.03983375, 0.7845332 ,
        0.20805988]], dtype=float32)

In [23]:
data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [49]:
t = test.as_numpy_iterator().next()

In [42]:
y_hat = model.predict(test)

997/997 ━━━━━━━━━━━━━━━━━━━━ 23s 22ms/step


In [48]:
y_hat[967]

array([8.8616402e-04, 3.6390273e-07, 4.6693403e-05, 2.9743003e-05,
       1.1932085e-04, 4.7758873e-05], dtype=float32)

In [60]:
t

(array([[ 1458,     2, 55148, ...,     0,     0,     0],
        [    8,    69,    12, ...,     0,     0,     0],
        [   65,    67,    65, ...,     0,     0,     0],
        ...,
        [   21,  2434,    34, ...,     0,     0,     0],
        [25229,     8,   590, ...,     0,     0,     0],
        [  202,  1740,   216, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [51]:
res = model.predict(t)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(16, 1000))', 'Tensor(shape=(16, 6))')
  warnings.warn(msg)


In [59]:
res

array([[3.66319320e-04, 1.02650844e-07, 1.83533048e-05, 1.25697425e-05,
        4.78683978e-05, 1.92625321e-05],
       [9.83442762e-04, 5.16885564e-07, 5.71782184e-05, 3.83677907e-05,
        1.40175034e-04, 5.90144118e-05],
       [2.54815310e-01, 7.70472689e-04, 2.58811638e-02, 4.63707419e-03,
        4.98145893e-02, 1.19818142e-02],
       [1.99538958e-03, 1.34764593e-06, 1.22325786e-04, 7.20523240e-05,
        2.91999924e-04, 1.20079101e-04],
       [4.46405757e-04, 1.20347622e-07, 2.11556544e-05, 1.39155818e-05,
        5.62693058e-05, 2.20214897e-05],
       [6.99010270e-05, 9.55576063e-09, 3.16024784e-06, 2.55556552e-06,
        8.48393120e-06, 3.42492399e-06],
       [3.74785741e-03, 3.33141816e-06, 2.40991416e-04, 1.33189169e-04,
        5.57198829e-04, 2.27757322e-04],
       [9.95243371e-01, 3.88768911e-01, 9.64298129e-01, 3.28374952e-02,
        8.30218136e-01, 2.17040673e-01],
       [1.10978093e-04, 1.60095119e-08, 4.69236375e-06, 3.61044022e-06,
        1.29767604e-05, 

In [63]:
res = (res > 0.5).astype(int)

In [71]:
from tensorflow.keras.metrics import Precision,Recall,Accuracy

In [69]:
pre = Precision()
re = Recall()

In [68]:
pre.update_state(t[1],res)
print(pre.result())
re.update_state(t[1],res)
print(f'recall {re.result()}')

tf.Tensor(1.0, shape=(), dtype=float32)
recall 0.75


In [72]:
acc = Accuracy()
acc.update_state(t[1],res)
print(acc.result())

tf.Tensor(0.9791666, shape=(), dtype=float32)
